In [2]:
import torch

In [3]:
t = torch.load("/Users/namle/Library/CloudStorage/GoogleDrive-hcmutepfl@gmail.com/My Drive/GET/saved_model/Politifacts/poli_0")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [1]:
import sys
# sys.path.insert(0, '../GET')


from Models.FCWithEvidences import graph_based_semantic_structure
from Fitting.FittingFC import char_man_fitter_query_repr1
import time
import json
from interactions import ClassificationInteractions
import matchzoo as mz
from handlers import cls_load_data
import argparse
import random
import numpy as np
import torch
import torch_utils
import numpy as np
import os
import datetime
from handlers.output_handler_FC import FileHandlerFC
from Evaluation import mzEvaluator as evaluator
from setting_keywords import KeyWordSettings
from matchzoo.embedding import entity_embedding
from Models.BiDAF.wrapper import GGNN, GGNN_with_GSL, Linear
from thirdparty.two_branches_attention import *
from new_model import *


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/testget"
loss_type="cross_entropy"
batch_size=32
num_folds=5
use_claim_source=0
use_article_source=1
path="formatted_data/declare/"
hidden_size=300
epochs=100
num_att_heads_for_words=5
num_att_heads_for_evds=2
gnn_window_size=3
lr=0.0001
gnn_dropout=0.2
seed=123456
gsl_rate=0.6
fixed_length_left_src_chars = 20
fixed_length_right_src_chars = 20
seed = 123456
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
preprocessor = mz.preprocessors.CharManPreprocessor(fixed_length_left = fixed_length_left,
                                                    fixed_length_right = fixed_length_right,
                                                    fixed_length_left_src = fixed_length_left_src_chars,
                                                    fixed_length_right_src = fixed_length_right_src_chars)

In [4]:
dataset="Snopes"
fixed_length_left=30
fixed_length_right=100
log="logs/get"
batch_size=32
gnn_window_size=3
fold_idx=0
if not os.path.exists(log):
      os.mkdir(log)
secondary_log_folder = os.path.join(log, "log_results_%s" % (dataset))
if not os.path.exists(secondary_log_folder):
  os.mkdir(secondary_log_folder)
secondary_log_folder = secondary_log_folder
root = os.path.join(os.path.join(path,dataset), "mapped_data")
tx = time.time()
kfold_dev_results, kfold_test_results = [], []
list_metrics = KeyWordSettings.CLS_METRICS
outfolder_per_fold = os.path.join(secondary_log_folder, "Fold_%s" % fold_idx)
if not os.path.exists(outfolder_per_fold):
    os.mkdir(outfolder_per_fold)
logfolder_result_per_fold = os.path.join(outfolder_per_fold, "result_%s.txt" % int(seed))
file_handler = FileHandlerFC()
file_handler.init_log_files(logfolder_result_per_fold)
# root ="/content/drive/MyDrive/GET/formatted_data/declare/Snopes/mapped_data"
# ------------------------------------------------------------------------------------
predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'test_%s' % 0, kfolds = num_folds)
train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'train_%sres' % 0, kfolds = num_folds)
# predict_pack = cls_load_data.load_data(root + "/%sfold" % num_folds,  'testnew' , kfolds = num_folds)
# train_pack = cls_load_data.load_data(root + "/%sfold" % num_folds, 'trainnew' , kfolds = num_folds)
valid_pack = cls_load_data.load_data(root, 'dev', kfolds = num_folds)

a = train_pack.left["text_left"].str.lower().str.split().apply(len).max()
b = valid_pack.left["text_left"].str.lower().str.split().apply(len).max()
c = predict_pack.left["text_left"].str.lower().str.split().apply(len).max()
max_query_length = max([a, b, c])
min_query_length = min([a, b, c])

a = train_pack.right["text_right"].str.lower().str.split().apply(len).max()
b = valid_pack.right["text_right"].str.lower().str.split().apply(len).max()
c = predict_pack.right["text_right"].str.lower().str.split().apply(len).max()
max_doc_length = max([a, b, c])
min_doc_length = min([a, b, c])

file_handler.myprint("Min query length, " + str(min_query_length) + " Min doc length " + str(min_doc_length))
file_handler.myprint("Max query length, " + str(max_query_length) + " Max doc length " + str(max_doc_length))
global additional_data
additional_data = {KeyWordSettings.OutputHandlerFactChecking: file_handler,
                    KeyWordSettings.GNN_Window: gnn_window_size}

print('parsing data')

train_processed = preprocessor.fit_transform(train_pack)  # This is a DataPack
valid_processed = preprocessor.transform(valid_pack)
predict_processed = preprocessor.transform(predict_pack)


train_interactions = ClassificationInteractions(train_processed, **additional_data)
valid_interactions = ClassificationInteractions(valid_processed, **additional_data)
predict_interactions = ClassificationInteractions(predict_processed, **additional_data)
file_handler.myprint('done extracting')
  

100%|██████████| 2756/2756 [00:00<00:00, 765580.62it/s]


Min query length, 22 Min doc length 100
Max query length, 28 Max doc length 100
parsing data


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 10421.84it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:07<00:00, 2874.46it/s]
Processing text_right with extend: 100%|██████████| 21074/21074 [00:00<00:00, 178510.22it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 3126/3126 [00:00<00:00, 11445.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 21074/21074 [00:06<00:00, 3066.78it/s]
Processing text_right with transform: 100%|██████████| 2756/2756 [00:00<00:00, 172313.84it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 433/433 [00:00<00:00, 11670.31it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2756/2756 [00:00<00:

Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id


/Users/namle/Desktop/GET/interactions.py:15: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
Converting DataFrame to Normal Dictionary of Data
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
[NOTICE] MatchZoo use queryID and docID as index in dataframe left and right, therefore, iterrows will return index which is left_id or right_id
done extracting


In [5]:
print("Loading word embeddings......")
t1_emb = time.time()
term_index = preprocessor.context['vocab_unit'].state['term_index']
glove_embedding = mz.datasets.embeddings.load_glove_embedding_FC(dimension = 300,
                                                                  term_index = term_index, **additional_data)

embedding_matrix = glove_embedding.build_matrix(term_index)
entity_embs1 = entity_embedding.EntityEmbedding(128)
claim_src_embs_matrix = entity_embs1.build_matrix(preprocessor.context['claim_source_unit'].state['term_index'])

entity_embs2 = entity_embedding.EntityEmbedding(128)
article_src_embs_matrix = entity_embs2.build_matrix(preprocessor.context['article_source_unit'].state['term_index'])

t2_emb = time.time()
print("Time to load word embeddings......", (t2_emb - t1_emb))

Loading word embeddings......
Word hit: (32823, 49881) 65.8026102123053
Time to load word embeddings...... 2.483272075653076


In [6]:
import datetime
from handlers.output_handler_FC import FileHandlerFC
import os
loss_type = 'cross_entropy'
epochs = 5
batch_size = 32
lr = 0.001
early_stopping = 10
outfolder_per_fold = "logs/get/log_results_PolitiFact/Fold_10"
curr_date = datetime.datetime.now().timestamp()
fixed_num_evidences = 30
seed = 123456
logfolder_result_per_fold = os.path.join(outfolder_per_fold, "result_%s.txt" % int(seed))
file_handler = FileHandlerFC()
file_handler.init_log_files(logfolder_result_per_fold)

match_model = PropagationModel()

# file_handler.myprint("Fitting Model")
fit_model = FittingModel(net = match_model,batch_size=batch_size, n_iter = epochs,idx_fold=0,threshold=0.8,data="snope",output_handler_fact_checking = file_handler,use_cuda=0,logfolder=outfolder_per_fold,seed=seed)



Using: <new_model.Loss_func object at 0x7fe262feb940>


In [ ]:
result_val = fit_model.evaluate(valid_interactions, 10,"valid")

In [8]:
result_val = fit_model.evaluate(predict_interactions, 10,type_set="test")

In [7]:
fit_model.fit(train_interactions)

AttributeError: 'NoneType' object has no attribute 'claims'

In [9]:
val_interactions = valid_interactions
test_interactions = predict_interactions

fit_model._net.train(True)
best_val_f1_macro, best_epoch = 0, 0
test_results_dict = None
iteration_counter = 0
count_patience_epochs = 0
epoch_loss, total_pairs = 0.0, 0
t1 = time.time()
for epoch_num in range(1):
    query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
    evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
    pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(train_interactions,100)

    del left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
    evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, evd_docs_adj
    gc.collect()
    torch.cuda.empty_cache()
    query_2_vector = {} 
    for i,id in enumerate(query_ids):
        query_2_vector[id] = i
    id_b = np.array([query_2_vector[i] for i in query_ids ])
    id_b,query_ids, pair_labels = my_utils.shuffle(id_b,query_ids, pair_labels)
    # all_labels = pair_labels.flatten().astype(int).tolist()

    # pair_labels = pair_labels.tolist()
    for (minibatch_num,
            (batch_id,batch_label)) \
            in enumerate(my_utils.minibatch(id_b,pair_labels, batch_size=fit_model._batch_size)):
        batch_label = my_utils.gpu(torch.from_numpy(batch_label), fit_model._use_cuda)
        batch_id = my_utils.gpu(torch.from_numpy(batch_id), fit_model._use_cuda)
        fit_model._optimizer.zero_grad()

        loss = fit_model._get_prediction("train",batch_id,batch_label)
        epoch_loss += loss.item()
        iteration_counter += 1
        # if iteration_counter % 2 == 0: break
        # TensorboardWrapper.mywriter().add_scalar("loss/minibatch_loss", loss.item(), iteration_counter)
        loss.backward()
        fit_model._optimizer.step()

        # TensorboardWrapper.mywriter().add_scalar("loss/epoch_loss_avg", epoch_loss, epoch_num)
        # print("Number of Minibatches: ", minibatch_num, "Avg. loss of epoch: ", epoch_loss)
        t2 = time.time()
        epoch_train_time = t2 - t1
    if True:  # validation after each epoch
            
            f1_macro_val = fit_model._output_results_every_epoch( val_interactions, test_interactions,10, epoch_num, epoch_train_time, epoch_loss)
            if f1_macro_val > best_val_f1_macro :
                # if (hits + ndcg) > (best_hit + best_ndcg):
                count_patience_epochs = 0
                with open(fit_model.saved_model, "wb") as f:
                    torch.save(fit_model._net.state_dict(), f)
                # test_results_dict = result_test
                best_val_f1_macro, best_epoch = f1_macro_val, epoch_num
                # test_hit, test_ndcg = hits_test, ndcg_test
            else:
                count_patience_epochs += 1
            # if fit_model._early_stopping_patience and count_patience_epochs > fit_model._early_stopping_patience:

            #     fit_model.output_handler.myprint(
            #         "Early Stopped due to no better performance in %s epochs" % count_patience_epochs)
            #     break

|Epoch 000 | Train time: 09.9(s) | Train loss: 0.896| Val F1_macro = 0.185 | Vad AUC = 0.136| Val F1_micro = 0.226 | Validation time: 86.4(s)
|Epoch 000 | Train time: 103.1(s) | Train loss: 1.808| Val F1_macro = 0.185 | Vad AUC = 0.135| Val F1_micro = 0.226 | Validation time: 89.0(s)
|Epoch 000 | Train time: 199.1(s) | Train loss: 2.727| Val F1_macro = 0.185 | Vad AUC = 0.137| Val F1_micro = 0.226 | Validation time: 87.8(s)
|Epoch 000 | Train time: 293.5(s) | Train loss: 3.626| Val F1_macro = 0.185 | Vad AUC = 0.137| Val F1_micro = 0.226 | Validation time: 89.4(s)
|Epoch 000 | Train time: 389.6(s) | Train loss: 4.558| Val F1_macro = 0.183 | Vad AUC = 0.139| Val F1_micro = 0.224 | Validation time: 88.5(s)
|Epoch 000 | Train time: 485.0(s) | Train loss: 5.519| Val F1_macro = 0.185 | Vad AUC = 0.137| Val F1_micro = 0.226 | Validation time: 90.7(s)


KeyboardInterrupt: 

In [10]:
test = torch.load('/Users/namle/Desktop/GET/embed_probagation/logit_snope_test_0.pt')

In [11]:
test.size()

torch.Size([782, 2])

In [16]:
print(torch.cuda.is_available())

False


In [15]:
test = my_utils.gpu(test, 1)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
test

tensor([[ 2.9740e+00, -2.9955e+00],
        [ 4.2828e+00, -4.4486e+00],
        [ 2.0279e+00, -1.9351e+00],
        [ 4.3739e+00, -4.4272e+00],
        [ 2.9412e+00, -3.0697e+00],
        [ 6.5988e+00, -6.5811e+00],
        [ 4.3376e+00, -4.5017e+00],
        [ 2.1097e+00, -2.0925e+00],
        [ 2.6977e+00, -2.6192e+00],
        [ 3.8715e-01, -5.3584e-01],
        [ 8.1147e-01, -1.0439e+00],
        [ 3.4172e+00, -3.4631e+00],
        [ 4.4300e+00, -4.6018e+00],
        [ 1.7734e+00, -1.9527e+00],
        [ 2.9736e+00, -3.0286e+00],
        [ 2.4441e+00, -2.5156e+00],
        [ 3.1239e+00, -3.3047e+00],
        [ 7.3578e+00, -7.1843e+00],
        [-3.7226e-01,  3.4136e-01],
        [ 2.2568e+00, -2.3130e+00],
        [-1.5962e-01, -5.0006e-03],
        [-1.3519e+00,  1.2971e+00],
        [ 4.0721e+00, -4.1618e+00],
        [ 5.0162e+00, -5.0266e+00],
        [ 7.5891e-01, -9.1844e-01],
        [ 6.6964e+00, -6.6833e+00],
        [ 9.5095e-01, -9.5208e-01],
        [ 3.1407e+00, -3.333

In [9]:
result_val

{'auc': 0.907009657009657,
 'f1_macro': 0.8088824538163746,
 'f1_micro': 0.8478260869565218,
 'f1': 0.7226107226107227,
 'precision_true_cls': 0.7077625570776256,
 'recall_true_cls': 0.7380952380952381,
 'f1_true_cls': 0.7226107226107227,
 'precision_false_cls': 0.9023090586145648,
 'recall_false_cls': 0.8881118881118881,
 'f1_false_cls': 0.8951541850220264,
 'precision_mixed_cls': 0.0,
 'recall_mixed_cls': 0.0,
 'f1_mixed_cls': 0.0}

In [9]:
query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
            evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
            pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(valid_interactions,100)

In [10]:
all_labels = pair_labels.flatten().astype(int).tolist()

In [13]:
query_2_vector = {}
pair_labels = pair_labels.tolist()
for i,id in enumerate(query_ids):
    query_2_vector[id] = i
id_b = np.array([query_2_vector[i] for i in query_ids ])

In [17]:
logits = torch.load("/Users/namle/Desktop/GET/embed_probagation/logit_snope_valid.pt")
probs = F.softmax(logits,dim=-1)

In [23]:
all_final_preds = []
all_final_probs = []
predictions = probs.argmax(dim=1)
# all_final_preds.append(float(my_utils.cpu(predictions).detach().numpy().flatten()))
# all_final_probs.append(float(my_utils.cpu(probs[:, 1]).detach().numpy().flatten()))
all_final_preds = all_final_preds + predictions.cpu().detach().numpy().flatten().astype(int).tolist()
all_final_probs = all_final_probs + logits[:, 1].cpu().detach().numpy().flatten().tolist()

In [24]:
results = fit_model._computing_metrics(true_labels=all_labels, predicted_labels=all_final_preds, predicted_probs=all_final_probs)

In [25]:
results

{'auc': 0.8718590231698031,
 'f1_macro': 0.7887235731503937,
 'f1_micro': 0.8452655889145496,
 'f1': 0.6794258373205742,
 'precision_true_cls': 0.7634408602150538,
 'recall_true_cls': 0.6120689655172413,
 'f1_true_cls': 0.6794258373205742,
 'precision_false_cls': 0.8676470588235294,
 'recall_false_cls': 0.9305993690851735,
 'f1_false_cls': 0.898021308980213,
 'precision_mixed_cls': 0.0,
 'recall_mixed_cls': 0.0,
 'f1_mixed_cls': 0.0}

In [18]:
t1 = np.arange(0,10,1)
t2 = np.arange(10,20,1)

In [21]:
t1, t2 = my_utils.shuffle(t1, t2)

In [8]:
all_final_preds = []
all_final_probs = []
query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
            evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
            pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(valid_interactions,100)
query_2_vector = {} 
for i,id in enumerate(query_ids):
    query_2_vector[id] = i
id_b = np.array([query_2_vector[i] for i in query_ids ])
id_b,query_ids, pair_labels = my_utils.shuffle(id_b,query_ids, pair_labels)
all_labels = pair_labels.flatten().astype(int).tolist()

pair_labels = pair_labels.tolist()

for (minibatch_num,
                (batch_id,batch_label)) \
                in enumerate(my_utils.minibatch(id_b,pair_labels, batch_size=fit_model._batch_size)):
        # padded_evd_contents = self._pad_evidences(evd_contents) # 1, 30, R
            # new_batch =  self.get_new_batch(type_set,batch_id)
        old_id,x,edge_index = fit_model.get_graph("valid",batch_id,fit_model._threshold)
        probs = fit_model._net.predict(x,edge_index,old_id)

        predictions = probs.argmax(dim=1)
        # all_final_preds.append(float(my_utils.cpu(predictions).detach().numpy().flatten()))
        # all_final_probs.append(float(my_utils.cpu(probs[:, 1]).detach().numpy().flatten()))
        all_final_preds = all_final_preds + predictions.cpu().detach().numpy().flatten().astype(int).tolist()
        all_final_probs = all_final_probs + probs[:, 1].cpu().detach().numpy().flatten().tolist()

KeyboardInterrupt: 

In [10]:
epoch_loss, total_pairs = 0.0, 0
iteration_counter = 0
for epoch_num in range(1):
    query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
    evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
    pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(valid_interactions,100)

    del left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
    evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, evd_docs_adj
    gc.collect()
    torch.cuda.empty_cache()
    query_2_vector = {} 
    for i,id in enumerate(query_ids):
        query_2_vector[id] = i
    id_b = np.array([query_2_vector[i] for i in query_ids ])
    id_b,query_ids, pair_labels = my_utils.shuffle(id_b,query_ids, pair_labels)
    # all_labels = pair_labels.flatten().astype(int).tolist()
    t1 = time.time()
    # pair_labels = pair_labels.tolist()
    for (minibatch_num,
            (batch_id,batch_label)) \
            in enumerate(my_utils.minibatch(id_b,pair_labels, batch_size=fit_model._batch_size)):
        batch_label = my_utils.gpu(torch.from_numpy(batch_label), fit_model._use_cuda)
        batch_id = my_utils.gpu(torch.from_numpy(batch_id), fit_model._use_cuda)
        fit_model._optimizer.zero_grad()

        loss = fit_model._get_prediction("valid",batch_id,batch_label)
        epoch_loss += loss.item()
        iteration_counter += 1
        t2 = time.time()
        pred_time = t2 - t1

        print(f'pred_time {pred_time}')
        print(f'Loss: {loss.item()}')
        break


pred_time 7.008730888366699
Loss: 0.9456478953361511


In [29]:
batch_id.size()

torch.Size([32])

In [30]:
batch_id.size()

torch.Size([32])

In [31]:
old_id,x,edge_index = fit_model.get_graph("valid",batch_id,fit_model._threshold)

In [22]:
len(old_id)

433

In [32]:
edge_index.size()

torch.Size([2, 8928])

In [19]:
similarity_dict = fit_model.test_similarity_dict

In [20]:
len(similarity_dict)

782

In [15]:
 query_ids, left_contents, left_lengths, query_sources, query_char_sources, query_adj, \
            evd_docs_ids, evd_docs_contents, evd_docs_lens, evd_sources, evd_cnt_each_query, evd_char_sources, \
            pair_labels, evd_docs_adj = fit_model._sampler.get_train_instances_char_man(predict_interactions,100)
        # query_ids, pair_labels = my_utils.shuffle(query_ids, pair_labels)
query_2_vector = {} 
for i,id in enumerate(query_ids):
    query_2_vector[id] = i
id_b = np.array([query_2_vector[i] for i in query_ids ])

In [17]:
id_b

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18